In [1]:
import numpy as np
import tensorflow as tf

In [2]:
'''
訓練データと正解ラベル
'''
train = np.array([[0, 0],  # 0と1の組み合わせの行列(4データ,2列)              
                  [0, 1],
                  [1, 0],
                  [1, 1]])
label = np.array([[0],     # 正解ラベル(4データ,1列)
                  [1],
                  [1],
                  [0]])

In [3]:
learning_rate = 0.1    # 学習率
training_epochs = 4000 # エポック数

In [4]:
# データフローグラフ

''' 
入力層(行数可変, 各データのサイズ＝2列)
''' 
xx = tf.placeholder(tf.float32, shape=[None, 2])

''' 
隠れ層（ニューロン数＝2
'''
# 重み(各データのサイズ＝2行, ニューロン数＝2列)
W1 = tf.Variable(tf.truncated_normal([2, 2]))
# バイアス(ニューロン数＝2列)
b1 = tf.Variable(tf.zeros([2]))
# 活性化関数:出力(入力データの行数＝4行, ニューロン数＝2列)
out1 = tf.nn.sigmoid(tf.matmul(xx, W1) + b1)

''' 
出力層（ニューロン数＝1）
''' 
# 重み(前層のニューロン数＝2行,クラス数＝ニューロン数＝1列)
W2 = tf.Variable(tf.truncated_normal([2, 1]))
# バイアス(クラス数＝ニューロン数＝1列)
b2 = tf.Variable(tf.zeros([1]))
# 活性化関数:出力((前層の出力行数＝4行, 2層のニューロン数＝1列)
out2 = tf.nn.sigmoid(tf.matmul(out1, W2) + b2)

'''
プレースホルダー
'''
# 正解ラベルを保持するプレースホルダー(行数可変, 列数は正解ラベルと同じ1)
t = tf.placeholder(tf.float32, shape=[None, 1])

''' 
ミニマイザーによる最適化
''' 
# 交差エントロピー誤差関数
cost =  - tf.reduce_sum(t * tf.log(out2) + (1 - t) * tf.log(1 - out2))
# ミニマイザー
train_op = tf.train.GradientDescentOptimizer(
    learning_rate              # 学習率
    ).minimize(cost)           # 最小化問題を解く


'''
変数の初期化ノード
'''
init_op = tf.global_variables_initializer()

'''
テストデータの予測を判定するopノード
'''
correct_prediction = tf.equal( # 要素ごとに(out2＝t)の真偽値を取得
    tf.to_float(tf.greater(    # (out2 > 0.5）の真偽値をfloatに変換
        out2,0.5)),
    t                          # 正解ラベル
    )

In [5]:
# セッション部
with tf.Session() as sess:
    sess.run(init_op)              # initノードを実行して変数を初期化

    for epoch in range(training_epochs):
        err, _ = sess.run(
            [cost,                 # 交差エントロピー誤差関数ノードを実行
             train_op],            # ミニマイザーを実行
             feed_dict={xx: train, # 訓練データのミニバッチ
                        t: label   # 正解ラベルのミニバッチ
                        }
            )

        epoch += 1                 # 処理回数として1加算
        if epoch % 500 == 0:       # 500回ごとに誤差を出力
            print('(',epoch, ')',err)

    print('(',epoch, ')',err)      # 学習終了時の誤差を出力
    print('Check:')
    print(
        correct_prediction.eval(   # 学習結果で予測判定する
            feed_dict={xx: train,  # 訓練データをセット
                       t: label    # 正解ラベルをセット
                      }
        ))
    print('Output Probability:')   # 最終出力を表示
    print(
        out2.eval(                 # out2ノード(活性化関数)を実行
            feed_dict={xx: train}
        ))

( 500 ) 2.4967477
( 1000 ) 0.50661135
( 1500 ) 0.1530146
( 2000 ) 0.0870377
( 2500 ) 0.060298152
( 3000 ) 0.04596528
( 3500 ) 0.037070498
( 4000 ) 0.031027146
( 4000 ) 0.031027146
Check:
[[ True]
 [ True]
 [ True]
 [ True]]
Output Probability:
[[0.00748971]
 [0.99000096]
 [0.9930019 ]
 [0.00640629]]
